# Atmospheric attenuation

In this notebook we have a look at the error in applying the atmospheric attenuation to a signal.

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from auraliser.propagation import apply_atmospheric_absorption
from acoustics import Signal
from acoustics.atmosphere import Atmosphere

In [2]:
from IPython.parallel import Client
client = Client()
lview = client.load_balanced_view()

/home/freddy/miniconda3/lib/python3.4/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


ImportError: No module named 'parallel'

In [ ]:
def calculate_signal(kwargs):
    
    def _calculate(fs, frequency, distance, n_blocks, n_distances, duration): 
        from auraliser.propagation import apply_atmospheric_absorption
        from acoustics import Signal
        from acoustics.atmosphere import Atmosphere
        import numpy as np
        atmosphere = Atmosphere()

        samples = int(fs*duration)
        t = np.arange(samples) / fs

        signal = Signal( np.sin(2.0*np.pi*frequency*t) , fs)

        out = Signal( apply_atmospheric_absorption(signal, fs, atmosphere, distance*np.ones(samples), 
                                                   n_blocks=n_blocks, n_distances=n_distances) , fs)

        att = atmosphere.attenuation_coefficient(frequency) * distance

        difference = signal.leq() - out.leq()
        return difference - att
    
    return _calculate(**kwargs)

## Parameters

In [ ]:
params = dict()
params['duration'] = 5.0
params['fs'] = 10025
params['frequency'] = 1000.0
params['distance'] = 100.0
params['n_blocks']= 512
params['n_distances'] = 10

In [ ]:
def study(params, parameter, values):
    """Parameter study."""
    for value in values:
        out = params.copy()
        out[parameter] = value
        yield out

## Vary duration

In [ ]:
#values = [5.0, 10.0, 15.0, 20.0, 25.0, 30.0]
values = [2.0, 4.0, 8.0, 16.0]
parameters = list( study(params, 'duration', values) )
out = lview.map(calculate_signal, parameters)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(values, out)
ax.set_title("Error in dB")

## Vary sample frequency

In [ ]:
values = [5000.0, 8000.0, 12000.0, 20000.0, 30000.0, 44100.0]
parameters = list( study(params, 'fs', values) )
out = lview.map(calculate_signal, parameters)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(values, out)
ax.set_title("Error in dB")

## Vary frequency

In [ ]:
values = [100.0, 200.0, 800.0, 1600.0, 3200.0, 6400.0]
parameters = list( study(params, 'fs', values) )
out = lview.map(calculate_signal, parameters)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(values, out)
ax.set_title("Error in dB")

## Vary distance

In [ ]:
values = [100.0, 200.0, 400.0, 800.0, 1600.0, 3200.0]
parameters = list( study(params, 'distance', values) )
out = lview.map(calculate_signal, parameters)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(values, out)
ax.set_title("Error in dB")

## Vary blocks

In [ ]:
values = [16, 32, 64, 128, 256, 512, 1024, 2048]
parameters = list( study(params, 'n_blocks', values) )
out = lview.map(calculate_signal, parameters)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(values, out)
ax.set_title("Error in dB")

## Vary amount of unique distances

In [ ]:
values = [2, 4, 8, 16, 32, 64]
parameters = list( study(params, 'n_distances', values) )
out = lview.map(calculate_signal, parameters)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(values, out)
ax.set_title("Error in dB")

In [ ]:
#lview.shutdown(restart=True)